In [65]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

In [66]:
def spar(x,f,pos,range):
    y=0*x
    for i in np.arange(0,np.size(x,1)):
        ii=np.random.permutation(np.size(x,0))
        nn=int(max(1,np.fix(np.size(x,0)*f)))
        if pos==1:
            for j in np.arange(0,nn,1):
                y[ii[j],i]=np.random.randint(1,range+1)

        if pos==-1:
            for j in np.arange(0,nn,1):
                y[ii[j],i]=-np.random.randint(1,range+1)

        if pos==0:
            for j in np.arange(0,nn,1):
                y[ii[j],i]=np.random.randint(1,range+1)*((np.random.rand()>0.5)*2-1)
            pin=0
            
            for j in np.arange(np.size(y,0)):
                if y[j,i]<0:
                    pin=1
                    break
            if pin==0:
                for j in np.arange(np.size(y,0)):
                    if y[j,i]>0:
                        y[j,i]=-y[j,i]
                        break
    return y

In [67]:
def GenomeMatrix(NumMet,NumGene):
    NumGeneIn=NumGene-NumMet/4
    S=np.zeros((NumMet,int(NumGeneIn+NumMet/4)))
    range=3
    Num1=int(NumMet/4)

    t=S[0:Num1,int(NumMet/4):int(NumGeneIn/4+NumMet/4)]
    S[0:Num1,int(NumMet/4):int(NumGeneIn/4+NumMet/4)]=spar(t,0.5,-1,3)

    t=S[Num1:2*Num1,int(NumMet/4):int(NumGeneIn/4+NumMet/4)]
    S[Num1:2*Num1,int(NumMet/4):int(NumGeneIn/4+NumMet/4)]=spar(t,0.5,1,3)

    t=S[Num1:2*Num1,int(NumGeneIn/4+NumMet/4):int(NumGeneIn*3/4+NumMet/4)]
    S[Num1:2*Num1,int(NumGeneIn/4+NumMet/4):int(NumGeneIn*3/4+NumMet/4)]=spar(t,0.8,0,3)
    
    t=S[Num1:2*Num1,int(NumGeneIn*3/4+NumMet/4):int(NumGeneIn+NumMet/4)]
    S[Num1:2*Num1,int(NumGeneIn*3/4+NumMet/4):int(NumGeneIn+NumMet/4)]=spar(t,0.5,-1,3)

    t=S[2*Num1:3*Num1,int(NumGeneIn*3/4+NumMet/4):int(NumGeneIn+NumMet/4)]
    S[2*Num1:3*Num1,int(NumGeneIn*3/4+NumMet/4):int(NumGeneIn+NumMet/4)]=spar(t,0.5,1,3)

    for i in np.arange(0,int(NumMet/4),1):
        S[i,i]=np.random.randint(1,4)
    return S

In [68]:
def PlasmidMatrix(NumMet,NumGene,RR):
    S=np.zeros((int(NumMet),int(NumGene)))
    range=3
    Num1=int(NumMet/4)

    t=S[Num1:2*Num1,0:int(NumGene)] 
    S[Num1:2*Num1,0:int(NumGene)]=spar(t,0.5,-1,3)

    t=S[Num1*2:Num1*3,0:int(NumGene*RR)]
    S[Num1*2:Num1*3,0:int(NumGene*RR)]=spar(t,0.5,1,3)

    t=S[Num1*3:Num1*4,int(NumGene*RR):int(NumGene)]
    S[Num1*3:Num1*4,int(NumGene*RR):int(NumGene)]=spar(t,0.5,1,3)

    return S

In [69]:
NumMet=80
Type234Genes=np.subtract([500,1000,2000],NumMet/4)
Replicates=20
PlasSizes=np.arange(0,400,20)

In [70]:
BURD=np.zeros((len(Type234Genes),Replicates,len(PlasSizes)))
alpha=2
PlasCap=np.zeros((len(Type234Genes),Replicates,len(PlasSizes)))
TotalSelection=np.zeros((len(Type234Genes),Replicates,len(PlasSizes)))
for sdf in np.arange(0,len(Type234Genes),1):
    Type234Gene=Type234Genes[sdf]
    ChroSize=int(Type234Gene+NumMet/4)
    for rty in np.arange(0,Replicates,1):
        print(sdf*Replicates+rty)
        ChroNetwork=GenomeMatrix(NumMet,ChroSize)
        NumFluxs=ChroSize+PlasSizes
        coef0=np.random.rand(int(max(NumFluxs)))
        GR=[]
        PGR=[]
        for i in np.arange(0,len(NumFluxs),1):
            if i==0:
                RR=0
            else:
                RR=np.divide(10,PlasSizes[i])
            #RR=0.1
            NumFlux=NumFluxs[i]
            coef=np.concatenate((coef0[0:ChroSize],alpha*coef0[ChroSize:NumFlux]),axis=0)
            CellNetwork=np.concatenate((ChroNetwork,PlasmidMatrix(NumMet,NumFlux-ChroSize,RR)),axis=1)
            b0=(0,1)
            Bounds=[b0]
            for j in np.arange(0,NumFlux-1,1):
                Bounds.append(b0)
            CC=np.concatenate((0*coef[0:int(3/4*Type234Gene+NumMet/4)],-coef[int(3/4*Type234Gene+NumMet/4):int(NumFlux)]),axis=0)
            Aeq=CellNetwork[0:int(NumMet/2),0:NumFlux]
            Beq=np.zeros(int(NumMet/2))
            #result = linprog(CC, A_ub=None, b_ub=None, A_eq=Aeq, b_eq=Beq, bounds=Bounds,x0=np.random.rand(NumFlux))
            result = linprog(CC, A_ub=None, b_ub=None, A_eq=Aeq, b_eq=Beq, bounds=Bounds,x0=None)
            tt=result.x
            terp1=sum(np.multiply(coef[int(3/4*Type234Gene+NumMet/4):ChroSize],tt[int(3/4*Type234Gene+NumMet/4):ChroSize]))
            terp2=alpha*sum(np.multiply(coef[ChroSize:int(ChroSize+RR*PlasSizes[i])],tt[ChroSize:int(ChroSize+RR*PlasSizes[i])]))
            terp3=sum(np.multiply(coef[int(ChroSize+RR*PlasSizes[i]):NumFlux],tt[int(ChroSize+RR*PlasSizes[i]):NumFlux]))
            GR=np.append(GR,terp1+terp2)
            PGR=np.append(PGR,terp3)
            BURD[sdf,rty,i]=1-GR[-1]/GR[0]
            PlasCap[sdf,rty,i]=GR[-1]/(GR[-1]+PGR[-1])
            TotalSelection[sdf,rty,i]=BURD[sdf,rty,i]+PlasCap[sdf,rty,i]
            


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [71]:
plt.figure(figsize=(2,2))
for i in np.arange(0,len(Type234Genes),1):
    mm=np.mean(TotalSelection[i,0:Replicates,0:len(PlasSizes)],axis=0)
    rr=np.std(TotalSelection[i,0:Replicates,0:len(NumFluxs)],axis=0)
    plt.errorbar(PlasSizes,mm,yerr=rr,marker='.',fillstyle='none',markersize=10)

plt.tick_params(direction='in')
plt.xlabel('plasmid size',fontsize=16)
#plt.ylim(0,1)
plt.ylabel('total selective pressure',fontsize=16)
plt.savefig('FigS2c.pdf')
plt.show()
